In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
max_length = 256

In [3]:
from transformers import AutoTokenizer

encoding_args = {
    "max_length" : max_length,
    "padding" : True,
    "truncation" : True,
    "return_tensors" : "pt"
}

tokenizer = AutoTokenizer.from_pretrained("/raid/m13519061/ta/facebook-absa/src/output/checkpoint-2510")

/home/m13519061/anaconda3/envs/absa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda:0


In [5]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("/raid/m13519061/ta/facebook-absa/src/output/checkpoint-2510")
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (w

In [6]:
import data_utils
from evaluation import compute_metrics, summary_score

catch_answer_fn = data_utils.AnswerCatcher().lego_absa
decoding_args = {
    "skip_special_tokens" : False
}

In [7]:
from tqdm import tqdm
from typing import List, Dict, Tuple

def generate_predictions(model,tokenizer,data,device=torch.device("cuda:0"),decoding_args:Dict={}) -> List[str]:
    # Predict
    model = model
    tokenizer = tokenizer
    tensor_predictions = []
    with torch.no_grad():
        for text in tqdm(data):
            input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
            tensor_predictions.extend(model.generate(input_ids=input_ids, pad_token_id=tokenizer.pad_token_id,eos_token_id=tokenizer.eos_token_id,max_length=max_length).cpu())
            input_ids = input_ids.cpu()
            # attention_mask = attention_mask.cpu()
    tensor_predictions = [[token for token in row if token != -100] for row in tensor_predictions]
    predictions = tokenizer.batch_decode(tensor_predictions,**decoding_args)
    predictions = [el for el in predictions]
    return predictions

In [8]:
from datasets import Dataset

test_path = "../data/absa/id/william/test.txt"

test_tasks = [
    {
        "paradigm" : "extraction",
        "se_order" : "oas",
        "method" : "lego_absa"
    }
]

test = data_utils.read_data(test_path)
test_ds = data_utils.data_gen(data=test, nt_se_order="aos", tasks=test_tasks, n_fold=1, algo="round_robin", shuffle=False)
test_ds = Dataset.from_list(test_ds)

100%|██████████| 1000/1000 [00:00<00:00, 16847.44it/s]


In [9]:
str_preds = generate_predictions(model, tokenizer, test_ds["input"], device, decoding_args)

100%|██████████| 995/995 [09:46<00:00,  1.70it/s]


In [10]:
str_preds = [el.replace("</s>",'').replace("<pad>",'') for el in str_preds]

In [11]:
oas_ext_preds = [catch_answer_fn(out, "oas", text) for out, text in zip(str_preds, test_ds["input"])]

In [12]:
targets = [catch_answer_fn(out, "oas", text) for out, text in zip(test_ds["output"], test_ds["input"])]

In [13]:
summary_score(oas_ext_preds,targets)

{'recall': 0.689374305812662,
 'precision': 0.8334065934065934,
 'f1_score': 0.7545787999888001}

In [16]:
imp_prompter = data_utils.ImputePrompter()

In [20]:
from copy import deepcopy

test_tasks = [
    {
        "paradigm" : "extraction",
        "se_order" : "oa",
        "method" : "lego_absa"
    }
]

test = data_utils.read_data(test_path)
test_ds = data_utils.data_gen(data=test, nt_se_order="aos", tasks=test_tasks, n_fold=1, algo="round_robin", shuffle=False)
test_ds = Dataset.from_list(test_ds)
str_preds = generate_predictions(model, tokenizer, test_ds["input"], device, decoding_args)
str_preds = [el.replace("</s>",'').replace("<pad>",'') for el in str_preds]
oa_ext_preds = [catch_answer_fn(out, "oa", text) for out, text in zip(str_preds, test_ds["input"])]
oa_test = [
    imp_prompter.lego_absa(el["text"],rt,"oas") for el,rt in zip(test,oa_ext_preds)
]
str_preds = generate_predictions(model, tokenizer, oa_test, device, decoding_args)
oas_imp_preds1 = [catch_answer_fn(out, "oas", text) for out, text in zip(str_preds, test_ds["input"])]
oas_com_preds1 = []
for el1, el2 in zip(oas_ext_preds,oas_imp_preds1):
    result = deepcopy(el1)
    for el in el2:
        if el not in result:
            result.append(el)
    oas_com_preds1.append(result)

100%|██████████| 1000/1000 [00:00<00:00, 16557.40it/s]


In [39]:
summary_score(oas_com_preds1,targets)

{'recall': 0.7515734912995187,
 'precision': 0.6239419588875453,
 'f1_score': 0.6818363782763758}

In [43]:
from copy import deepcopy

test_tasks = [
    {
        "paradigm" : "extraction",
        "se_order" : "as",
        "method" : "lego_absa"
    }
]

test = data_utils.read_data(test_path)
test_ds = data_utils.data_gen(data=test, nt_se_order="aos", tasks=test_tasks, n_fold=1, algo="round_robin", shuffle=False)
test_ds = Dataset.from_list(test_ds)
str_preds = generate_predictions(model, tokenizer, test_ds["input"], device, decoding_args)
str_preds = [el.replace("</s>",'').replace("<pad>",'') for el in str_preds]
as_ext_preds = [catch_answer_fn(out, "as", text) for out, text in zip(str_preds, test_ds["input"])]
as_test = [
    imp_prompter.lego_absa(el["text"],rt,"oas") for el,rt in zip(test,as_ext_preds)
]
str_preds = generate_predictions(model, tokenizer, as_test, device, decoding_args)
oas_imp_preds2 = [catch_answer_fn(out, "oas", text) for out, text in zip(str_preds, test_ds["input"])]
oas_com_preds2 = []
for el1, el2 in zip(oas_ext_preds,oas_imp_preds2):
    result = deepcopy(el1)
    for el in el2:
        if el not in result:
            result.append(el)
    oas_com_preds2.append(result)

100%|██████████| 995/995 [13:12<00:00,  1.26it/s]


In [44]:
summary_score(oas_com_preds2,targets)

{'recall': 0.699740836727138,
 'precision': 0.38349611321506877,
 'f1_score': 0.49545557166792453}